
# Projet 2

In [7]:
%load_ext pycodestyle_magic

In [113]:
%flake8_off

In [9]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [14]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                  sep=';',
                  usecols=range(11),
                  index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320


7:80: E501 line too long (92 > 79 characters)


**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [15]:
geo['CP Ville'] = geo['Code Postal'] + " " + geo['Commune']

In [32]:
import math


def deg2rad(dd):
    """Convertit un angle "degrés décimaux" en "radians"
    """
    return dd/180*pi


# fonction recherche de ville
def ville(lat, long):

    # Rayon de la terre en mètres (sphère IAG-GRS80)
    RT = 6378137

    liste = []
    for i in range(geo.shape[0]):
        # angle en radians entre les 2 points
        S = acos(sin(lat)*sin(geo['Latitude'].iloc[i])
                 + cos(lat)*cos(geo['Latitude'].iloc[i])
                 * cos(abs(geo['Longitude'].iloc[i]-long)))
        liste.append(S*RT)  # distance sur un arc de grand cercle
    return geo['CP Ville'].iloc[liste.index(min(liste))]

In [34]:
ville(45.960848, 5.372926)

'01500 AMBERIEU-EN-BUGEY'

In [35]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [39]:
# on applique la fonction à une coordonnée tirée au hasard
# init du random
np.random.seed(0)

a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

'71330 BOSJEAN'

In [40]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'91120 PALAISEAU'

**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [48]:
import json
y = geo.loc["01001", "geo_shape"]
d = json.loads(y)
d

{'type': 'Polygon',
 'coordinates': [[[4.92627342692396, 46.1200517234555],
   [4.92199538031264, 46.12069170563269],
   [4.921996260482057, 46.12663667307488],
   [4.90936269187648, 46.129164756530656],
   [4.916016986429157, 46.14461766984426],
   [4.903080786126749, 46.157977074275074],
   [4.904571300671208, 46.16096059498421],
   [4.910603091203733, 46.180432597608316],
   [4.914119126867876, 46.18271538441311],
   [4.932895432675545, 46.18304616431434],
   [4.938729478686701, 46.17383114520867],
   [4.933038591703602, 46.16429901579509],
   [4.958286230120842, 46.153246666893935],
   [4.943330383388636, 46.145964148557894],
   [4.939154593056283, 46.12938048470421],
   [4.931056072485923, 46.12087123131487],
   [4.92627342692396, 46.1200517234555]]]}

In [68]:
d['type']

'Polygon'

In [80]:
print(len(d['coordinates']))
print(len(d['coordinates'][0]))
print(len(d['coordinates'][0][0]))

1
17
2


In [96]:
# Méthode 1 : liste

liste = []
for i in range(geo.shape[0]):
    d = json.loads(geo["geo_shape"].iloc[i])
    liste.append(len(d['coordinates'][0]))

print("La commune la plus complexe géométriquement est : " +
      geo['CP Ville'].iloc[liste.index(max(liste))])
print("La taille de son polygone est : " +
      str(len(json.loads(geo["geo_shape"]
                         .iloc[liste.index(max(liste))])['coordinates'][0])))

La commune la plus complexe géométriquement est : 97370 MARIPASOULA
La taille de son polygone est : 486


In [117]:
# Méthode 2 : dictionnaire

dict_pol = {}
for i in range(geo.shape[0]):
    dict_pol[geo['CP Ville'].iloc[i]] = len(json.loads(geo["geo_shape"].iloc[i])['coordinates'][0])
max_key = max(dict_pol, key=dict_pol.get)

print("La commune la plus complexe géométriquement est : " +
      max_key)
print("La taille de son polygone est : " +
      str(dict_pol[max_key]))

La commune la plus complexe géométriquement est : 97370 MARIPASOULA
La taille de son polygone est : 486


In [120]:
# 2ème commune la plus complexe géométriquement

dict_pol_sup = dict_pol.copy()
dict_pol_sup.pop(max_key)

max_key_2 = max(dict_pol_sup, key=dict_pol_sup.get)

print("La 2ème commune la plus complexe géométriquement est : " +
      max_key_2)
print("La taille de son polygone est : " +
      str(dict_pol_sup[max_key_2]))

La 2ème commune la plus complexe géométriquement est : 97390 REGINA
La taille de son polygone est : 392


In [131]:
liste_2 = []
for i in range(geo.shape[0]):
    d = json.loads(geo["geo_shape"].iloc[i])
    if len(d['coordinates']) == 2:
        liste_2.append(geo['CP Ville'].iloc[i])
print("Il y a " + str(len(liste_2)) + " villes multi-polygones :")
liste_2

Il y a 80 villes multi-polygones :


['02600 COYOLLES',
 '02110 SEBONCOURT',
 '04330 SENEZ',
 '05600 EYGLIERS',
 '06400/06150 CANNES',
 '06430 LA BRIGUE',
 '07320 SAINT-AGREVE',
 '07690 SAINT-ANDRE-EN-VIVARAIS',
 '08700 NOUZONVILLE',
 '08430 SINGLY',
 '09240 LA BASTIDE-DE-SEROU',
 '11160 PEYRIAC-MINERVOIS',
 '12740 SEBAZAC-CONCOURES',
 '15700 PLEAUX',
 '22560 PLEUMEUR-BODOU',
 '22650 PLOUBALAY',
 '27230 FOLLEVILLE',
 '27930 GRAVIGNY',
 '27400 INCARVILLE',
 '27100 LE VAUDREUIL',
 '28170 TREMBLAY-LES-VILLAGES',
 '29560 ARGOL',
 '29217 LE CONQUET',
 '29120 SAINT-JEAN-TROLIMON',
 '20169 BONIFACIO',
 '20100 SARTENE',
 '20214 CALENZANA',
 '33380 BIGANOS',
 '35750 IFFENDIC',
 '35320 TRESBOEUF',
 '39120 CHENE-BERNARD',
 '39140 LARNAUD',
 '39270 ORGELET',
 '42380 LA CHAPELLE-EN-LAFAYE',
 '42000/42100/42230 SAINT-ETIENNE',
 '45270 AUVILLIERS-EN-GATINAIS',
 '45270 BELLEGARDE',
 '45300 PANNECIERES',
 '48300 LANGOGNE',
 '48300 NAUSSAC',
 '50170 LE MONT-SAINT-MICHEL',
 '50200 SAINT-PIERRE-DE-COUTANCES',
 '51500 SAINT-LEONARD',
 '53240 

In [176]:
import shapely
from shapely.geometry.polygon import Polygon

In [199]:
for k in range(len(liste_2)):
    d = json.loads(geo[geo['CP Ville'] == liste_2[k]].geo_shape.iloc[0])
    
    try:
        pol1 = Polygon(d['coordinates'][0][0])
        pol2 = Polygon(d['coordinates'][1][0])
    except:
        pol1 = Polygon(d['coordinates'][0])
        pol2 = Polygon(d['coordinates'][1])

    if pol1.contains(pol2)==True:
        print(liste_2[k] + " OK")
    else:
        if pol2.contains(pol1)==True:
            print(liste_2[k] + " OK")

02600 COYOLLES OK
05600 EYGLIERS OK
09240 LA BASTIDE-DE-SEROU OK
22650 PLOUBALAY OK
20100 SARTENE OK
20214 CALENZANA OK
48300 LANGOGNE OK
67920 SUNDHOUSE OK
68420 EGUISHEIM OK
71270 BEAUVERNOIS OK
